<a href="https://colab.research.google.com/github/fkatelyn/colab/blob/master/palm_tree_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Palm Oil Tree detection training version 7

Image Size = 640x640
Model Size = Small
Epoch = 200



In [60]:
MODEL_VERSION = '7'

In [61]:
!nvidia-smi

Tue Sep 19 21:49:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [62]:
import os
HOME = os.getcwd()
HOME = '/content'
print(HOME)

from google.colab import drive
GOOGLE_DRIVE = f'{HOME}/drive'
drive.mount(GOOGLE_DRIVE)

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
class Experiment:
  '''
  Google drive path:

  PROJECT_DRIVE = MyDrive/ai/projects/palmtree
  EXPERIMENT_DRIVE = MyDrive/ai/projects/palmtree/experiments
  UPLOAD_DRIVE = MyDrive/ai/projects/palmtree/uploads
  DATASET_DRIVE = MyDrive/ai/projects/palmtree/datasets
  '''
  EPOCHS = 200
  IMGSZ = 640
  MODEL_NAME = "yolov8s"
  EXPERIMENT_HOME = f'{HOME}/experiments/{MODEL_VERSION}'
  DATASET_HOME = f'{HOME}/datasets/palm-tree-detection-{MODEL_VERSION}'

  PROJECT_DRIVE = f'{GOOGLE_DRIVE}/MyDrive/ai/projects/palmtree'
  EXPERIMENT_DRIVE = f'{PROJECT_DRIVE}/experiments/{MODEL_VERSION}'
  DATASET_DRIVE = f'{PROJECT_DRIVE}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_DRIVE = f'{PROJECT_DRIVE}/weights/{MODEL_VERSION}'


print(Experiment.EXPERIMENT_HOME)


/content/experiments/7


In [64]:

!pip install ultralytics==8.0.134

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.5/166.8 GB disk)


In [65]:
%cd {HOME}
%mkdir -p {Experiment.EXPERIMENT_HOME}
%mkdir -p {Experiment.DATASET_HOME}
%mkdir -p {Experiment.EXPERIMENT_DRIVE}
%mkdir -p {Experiment.WEIGHTS_DRIVE}

!cp -fpR {Experiment.DATASET_DRIVE} /content/datasets/
!ls /content

/content
datasets  drive  experiments


In [66]:
from ultralytics import YOLO

from IPython.display import display, Image

### 🐍 Python SDK

The simplest way of simply using YOLOv8 directly in a Python environment.

In [67]:
#model = YOLO(f'{HOME}/yolov8s.pt')
#results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

In [68]:
#results[0].boxes.xyxy

In [69]:
#results[0].boxes.conf

In [70]:
#results[0].boxes.cls

## Custom Training

In [71]:
class Dataset:
  def __init__(self, location):
    self.location = location


dataset = Dataset(location=Experiment.DATASET_HOME)
!rm -fR {Experiment.EXPERIMENT_HOME}/runs
!mkdir -p {Experiment.EXPERIMENT_HOME}/runs



In [ ]:

%cd {Experiment.EXPERIMENT_HOME}
!yolo task=detect mode=train model={Experiment.MODEL_NAME}.pt data={dataset.location}/data.yaml epochs={Experiment.EPOCHS} imgsz={Experiment.IMGSZ} plots=True

/content/experiments/7
New https://pypi.org/project/ultralytics/8.0.182 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/datasets/palm-tree-detection-7/data.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visu

In [ ]:
TRAIN_NUMBER = ''
!ls {Experiment.EXPERIMENT_HOME}/runs/detect/train{TRAIN_NUMBER}/

In [ ]:
%cd {Experiment.EXPERIMENT_HOME}
Image(filename=f'{Experiment.EXPERIMENT_HOME}/runs/detect/train{TRAIN_NUMBER}/confusion_matrix.png', width=600)

In [ ]:
%cd {Experiment.EXPERIMENT_HOME}
Image(filename=f'{Experiment.EXPERIMENT_HOME}/runs/detect/train{TRAIN_NUMBER}/results.png', width=600)

In [ ]:
%cd {Experiment.EXPERIMENT_HOME}
Image(filename=f'{Experiment.EXPERIMENT_HOME}/runs/detect/train{TRAIN_NUMBER}/val_batch0_pred.jpg', width=600)

## Validate Custom Model

In [ ]:
%cd {Experiment.EXPERIMENT_HOME}

!yolo task=detect mode=val model={Experiment.EXPERIMENT_HOME}/runs/detect/train{TRAIN_NUMBER}/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model

In [ ]:
%cd {Experiment.EXPERIMENT_HOME}
!yolo task=detect mode=predict model={Experiment.EXPERIMENT_HOME}/runs/detect/train{TRAIN_NUMBER}/weights/best.pt conf=0.30 source={dataset.location}/test/images save=True




Inspect results.

In [ ]:
PREDICT_NUMBER = ''
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{Experiment.EXPERIMENT_HOME}/runs/detect/predict{PREDICT_NUMBER}/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

## Saving the model weight to Google Drive

Store the best weight to Google Drive
/content/drive/MyDrive/ai/projects/palmtree/experiments/

In [ ]:
!ls -altr {Experiment.EXPERIMENT_HOME}/runs/detect/train/weights
!rm -fR {Experiment.EXPERIMENT_DRIVE}
!mkdir -p {Experiment.EXPERIMENT_DRIVE}
!cp -fpR {Experiment.EXPERIMENT_HOME}/runs {Experiment.EXPERIMENT_DRIVE}
!cp {Experiment.EXPERIMENT_HOME}/runs/detect/train/weights/best.pt {Experiment.WEIGHTS_DRIVE}